In [ ]:
#@title ドライブをマウントする
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title 出力ディレクトリを指定
cwd = "/content/drive/MyDrive/RNAseq/Outputs/\u5B9F\u9A13\u540D" #@param {type:"string"}
cwd = cwd + "/"

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
#@title ファイル名に使う実験名を定義する
exp_name="\u5B9F\u9A13\u540D\u3092\u5165\u529B\u3059\u308B" #@param {type:"string"}
exp_name = exp_name + "_"

In [ ]:
# 作成したデータパスを入力
df_tpm_path =  "\u5B9F\u9A13\u540D_avetpm.csv\u30D5\u30A1\u30A4\u30EB\u306E\u30D1\u30B9" #@param {type:"string"}
df_tpm = pd.read_csv(df_tpm_path , index_col = 0)
df_tpm

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
GlycoMaple用データの作成

In [ ]:
#@title glycan-related_genes.csv の読み込み、GlycoMaple用データの抽出
df_glyco_path = "/content/drive/MyDrive/RNAseq/SourceFiles/glycan-related_genes.csv" #@param {type:"string"}
df_glyco = pd.read_csv(df_glyco_path)
df_glyco_data = pd.merge(df_glyco , df_tpm)
# print(df_glyco_data)
# df_glyco_data.to_csv(exp_name + "for_GlycoMaple.csv" , sep = ",")

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
MetaScape用データの作成

In [ ]:
#@title データの確認

df_list = pd.DataFrame(list(df_tpm) , list(range(1 , len(list(df_tpm)) + 1 , 1))).T
df_list

In [ ]:
%%capture
#@title 平均列を比較し、発現変動遺伝子(DEG)を抽出
#@markdown コントロール群の平均列が何列目かを入力
Cont_ave_column = "\u30B3\u30F3\u30C8\u30ED\u30FC\u30EB\u7FA4\u306E\u5E73\u5747\u5217\u756A\u53F7" #@param {type:"string"}
Cont_num = int(Cont_ave_column) - 1
#@markdown テスト群の平均列が何列目かを入力
Test_ave_column = "\u30C6\u30B9\u30C8\u7FA4\u306E\u5E73\u5747\u5217\u756A\u53F7" #@param {type:"string"}
Test_num = int(Test_ave_column) - 1

df_ave = df_tpm.iloc[:, [Cont_num , Test_num]]

#@markdown 新しい列「DEG」を追加し(コントロール+1)/(テスト+1)の値を計算<br>
#@markdown 増減の理解しやすさのためにlog2の行も追加<br>
#@markdown 列「DEG」の値に基づいてソート

df_ave.loc[: , "DEG"] = (df_ave.iloc[:,1].values + 1) / (df_ave.iloc[:,0].values + 1)
df_ave.loc[: , "DEG_log2"] = np.log2(df_ave.loc[: , "DEG"])
df_ave.sort_values("DEG" , axis = 0 , ascending=False , inplace = True)
df_ave

In [ ]:
#@title 発現変動遺伝子を決定する
#@markdown 発現変動とする基準値を入力(X倍, 1/X倍を基準とする時のX)
DEG_sta = "2" #@param {type:"string"}
sta = float(DEG_sta)
sta = np.log2(sta)

#@markdown 発現変動遺伝子を増加と減少に分けてそれぞれcsvで保存する

df_DEG_up = df_ave[df_ave["DEG_log2"] >= sta]
df_DEG_down = df_ave[df_ave["DEG_log2"] <= -sta]

print("増加：%d" % len(df_DEG_up))
print("減少：%d" % len(df_DEG_down))

df_DEG_up.to_csv(exp_name + "発現" + DEG_sta + "倍増加遺伝子.csv" , sep = ",")
df_DEG_down.to_csv(exp_name + "発現" + DEG_sta + "分の1倍減少遺伝子.csv" , sep = ",")

In [ ]:
#@title TPMが微小なものは切り捨てる
#@markdown TPMがこの値未満のものは切り捨てる
drop_sta = "1" #@param {type:"string"}
drop_sta = float(drop_sta)

df_ave_drop = df_ave.copy()
df_ave_drop = df_ave_drop.reset_index()
df_ave_drop = df_ave_drop.set_index(["Geneid" , "DEG" , "DEG_log2"])
#df_ave_drop[df_ave_drop.mean() >= drop_sta]

for i in list(df_ave_drop):
  df_ave_drop = df_ave_drop[df_ave_drop[i] >= drop_sta]

df_ave_drop = df_ave_drop.reset_index()
df_ave_drop = df_ave_drop.set_index("Geneid")
df_DEG_up_drop = df_ave_drop[df_ave_drop["DEG_log2"] >= sta]
df_DEG_down_drop = df_ave_drop[df_ave_drop["DEG_log2"] <= -sta]

print("増加：%d" % len(df_DEG_up_drop))
print("減少：%d" % len(df_DEG_down_drop))

df_DEG_up_drop.to_csv(exp_name + "発現" + DEG_sta + "倍増加遺伝子" + str(drop_sta) + "未満TPM切り捨て" + ".csv" , sep = ",")
df_DEG_down_drop.to_csv(exp_name + "発現" + DEG_sta + "分の1倍減少遺伝子" + str(drop_sta) + "未満TPM切り捨て" + ".csv" , sep = ",")

In [ ]:
#@title 発現変動値を「実験名_DEGavetpm.csv」ファイルに保存
df_ave = df_ave.loc[: , ["DEG" , "DEG_log2"]]
df_ave.reset_index(inplace=True)
df_tpm.reset_index(inplace=True)
df_tpm_deg = pd.merge(df_tpm , df_ave , how = "inner")
# df_tpm_deg = df_tpm_deg.drop("index" , axis = 1)
df_tpm_deg = df_tpm_deg.set_index(["Geneid" , "product"])
df_tpm_deg = df_tpm_deg.drop_duplicates()
df_tpm_deg.to_csv(exp_name + "DEGavetpm.csv" , sep = ",")
df_tpm_deg